# Assignment 2

## 1. Loading packages and data

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import re
from nltk.stem import PorterStemmer
import nltk
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from gensim.models import Word2Vec

nltk.download('stopwords')
nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
from nltk.corpus import stopwords

pd.set_option('display.max_colwidth', None)  # Allows columns to display their full content so we can read the full reviews
pd.set_option('display.width', 200) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kkasp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\kkasp\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


In [19]:
file_path = r'IMDB Dataset.csv'
data = pd.read_csv(file_path)
print(data.head(20))


In [20]:
X = data['review']

def transform_bin(x):
  if x == "positive":
    return 1
  else:
    return 0
data['sentiment'] = data['sentiment'].apply(lambda x : transform_bin(x))

y = data['sentiment']
sentiment_counts = y.value_counts()
print("Class distribution in 'sentiment':\n", sentiment_counts)
print("50/50 class distribution")

Class distribution in 'sentiment':
 sentiment
1    25000
0    25000
Name: count, dtype: int64
50/50 class distribution


In [21]:
# Data split (80% training, 20% validation)
train_data, X_test, train_label, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(train_data, train_label, test_size=0.2, random_state=42)

X_train = X_train.to_frame()
X_test = X_test.to_frame()
X_val = X_val.to_frame()

# Display the sizes of the datasets
print("Training Data Size:", len(X_train))
print("Validation Data Size:", len(X_val))
print("Test Data Size:", len(X_test))

Training Data Size: 32000
Validation Data Size: 8000
Test Data Size: 10000


In [22]:
# Training set distribution
train_counts = y_train.value_counts()
print("Class Distribution in Training Set:\n", train_counts)

# Validation set distribution
val_counts = y_val.value_counts()
print("\nClass Distribution in Validation Set:\n", val_counts)

# Test set distribution
test_counts = y_test.value_counts()
print("\nClass Distribution in Test Set:\n", test_counts)

Class Distribution in Training Set:
 sentiment
0    16080
1    15920
Name: count, dtype: int64

Class Distribution in Validation Set:
 sentiment
1    4041
0    3959
Name: count, dtype: int64

Class Distribution in Test Set:
 sentiment
1    5039
0    4961
Name: count, dtype: int64


## 2. Text pre-processing 

In [23]:
def clean_text(text):
    # Remove unusual or non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenize the text into words for processing
    words = text.split()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]


    return filtered_words

X_train['cleaned_review'] = X_train['review'].apply(clean_text)
X_val['cleaned_review']  = X_val['review'].apply(clean_text)
X_test['cleaned_review']  = X_test['review'].apply(clean_text)


# Display the first few cleaned reviews
print("Cleaned Reviews with Stopwords Removed:")
print(X_train['cleaned_review'].head(5))

Cleaned Reviews with Stopwords Removed:
11794                                                                                                                                                                                                                                                           [fault, actors, put, great, performances, overall, story, well, executed, movie, opens, great, zinger, crazy, old, guy, forces, young, Aborigine, girls, car, road, forced, endure, 40, minutes, character, development, entirely, new, group, characters, dont, know, 40, minutes, turns, ones, eventually, discover, girls, body, story, progresses, therebr, br, story, pick, point, really, goes, nowhere, 2, hours, asked, point, see, characters, struggle, accusations, racism, stupidity, handled, discovery, story, ultimately, unsatisfying, felt, unfinished, well, acted, theres, strong, enough, backbone, film, warrant, recommending]
24925                                                                      

## 3. Further preprocessing: Features and Embeddings

#### Features

Here we create: Number of Positive lexicon words, Number of Negative lexicon words, Number of nos, and Number of words. After that the values were standardized

In [24]:
# 1. Lexicon Creation
positive_lexicon = set(opinion_lexicon.positive()) #list of positive words from the NLTK package
negative_lexicon = set(opinion_lexicon.negative()) #list of negative words from the NLTK package

# 2. Defining Functions
def counts_lex(review, type = 'neg'):
    count_neg = 0
    count_pos = 0
    for word in review:
      if word.lower() in positive_lexicon:
        count_pos += 1
      if word.lower() in negative_lexicon:
        count_neg += 1
    if type == 'neg':
      return count_neg
    elif type == 'pos':
      return count_pos

# Ensure reviews are tokenized, we use the review column as 'no' and 'not' gets removed by stopwords
def count_nos(review):
    count = 0
    words = review.split()  # Tokenize if review column is a single string
    for word in words:
        if word.lower() == "no" or word.lower() == "not":
            count += 1
    return count

# 3. Feature Extraction for Training, Validation, and Test Sets
X_train['NrPos'] = X_train['cleaned_review'].apply(lambda x : counts_lex(x, type = "pos"))
X_train['NrNeg'] = X_train['cleaned_review'].apply(lambda x : counts_lex(x, type = "neg"))
X_train['NrWords'] = X_train['cleaned_review'].apply(lambda x: len(x))
X_train['Nos'] = X_train['review'].apply(lambda x : count_nos(x))

X_val['NrPos'] = X_val['cleaned_review'].apply(lambda x : counts_lex(x, type = "pos"))
X_val['NrNeg'] = X_val['cleaned_review'].apply(lambda x : counts_lex(x, type = "neg"))
X_val['NrWords'] = X_val['cleaned_review'].apply(lambda x: len(x))
X_val['Nos'] = X_val['review'].apply(lambda x : count_nos(x))

X_test['NrPos'] = X_test['cleaned_review'].apply(lambda x : counts_lex(x, type = "pos"))
X_test['NrNeg'] = X_test['cleaned_review'].apply(lambda x : counts_lex(x, type = "neg"))
X_test['NrWords'] = X_test['cleaned_review'].apply(lambda x: len(x))
X_test['Nos'] = X_test['review'].apply(lambda x : count_nos(x))

# 4. Standardizing Features
cols = ['NrPos', 'NrNeg', 'NrWords', 'Nos']
train_features_to_transform = X_train[cols]
val_features_to_transform = X_val[cols]
test_features_to_transform = X_test[cols]

scaler = StandardScaler()
X_train_std = scaler.fit_transform(train_features_to_transform)  # X is your feature matrix
X_val_std = scaler.transform(val_features_to_transform)
X_test_std = scaler.transform(test_features_to_transform)

X_train[cols] = X_train_std
X_val[cols] = X_val_std
X_test[cols] = X_test_std

# 5. Stemming Words
# Apply stemming
stemmer = PorterStemmer()

X_train['Stem Words'] = X_train['cleaned_review'].apply(lambda x : [stemmer.stem(word.lower()) for word in x])
X_val['Stem Words'] = X_val['cleaned_review'].apply(lambda x : [stemmer.stem(word.lower()) for word in x])
X_test['Stem Words'] = X_test['cleaned_review'].apply(lambda x : [stemmer.stem(word.lower()) for word in x])

# 6. Output for debugging
# Display the first few cleaned reviews
print("Training Data with cleaned reviews with Stemming and New Features:")
print(X_train['cleaned_review'].head(20))

Training Data with cleaned reviews with Stemming and New Features:
11794                                                                                                                                                                                                                                                           [fault, actors, put, great, performances, overall, story, well, executed, movie, opens, great, zinger, crazy, old, guy, forces, young, Aborigine, girls, car, road, forced, endure, 40, minutes, character, development, entirely, new, group, characters, dont, know, 40, minutes, turns, ones, eventually, discover, girls, body, story, progresses, therebr, br, story, pick, point, really, goes, nowhere, 2, hours, asked, point, see, characters, struggle, accusations, racism, stupidity, handled, discovery, story, ultimately, unsatisfying, felt, unfinished, well, acted, theres, strong, enough, backbone, film, warrant, recommending]
24925                                           

In [25]:
# Save the preprocessed data in a csv file to save time if we want to re-run only the following part
X_train.to_csv("X_train.csv", index=False)
X_val.to_csv("X_val.csv", index=False)
X_test.to_csv("X_test.csv", index=False)
X_train

,review,cleaned_review,NrPos,NrNeg,NrWords,Nos,Stem Words
11794,"With no fault to the actors (they all put on great performances), the overall story was not very well executed. The movie opens with a great zinger: a crazy old guy forces a young Aborigine girl's car off the road. But then, we're forced to endure 40 minutes of character development with an entirely new group of characters ... and we don't know why until the 40 minutes are up. It turns out that they are the ones who eventually discover the girl's body ... and the story progresses from there.<br /><br />While the story does pick up at that point, it really goes nowhere. After 2 hours, I asked myself: was there a point to this, or was it just to see the characters struggle with accusations of racism and stupidity of how they handled the discovery? The story was ultimately unsatisfying and felt unfinished. While it is well acted, there's not a strong enough backbone in the film to warrant recommending it.","[fault, actors, put, great, performances, overall, story, well, executed, movie, opens, great, zinger, crazy, old, guy, forces, young, Aborigine, girls, car, road, forced, endure, 40, minutes, character, development, entirely, new, group, characters, dont, know, 40, minutes, turns, ones, eventually, discover, girls, body, story, progresses, therebr, br, story, pick, point, really, goes, nowhere, 2, hours, asked, point, see, characters, struggle, accusations, racism, stupidity, handled, discovery, story, ultimately, unsatisfying, felt, unfinished, well, acted, theres, strong, enough, backbone, film, warrant, recommending]",-0.349058,-0.255423,-0.481820,0.761786,"[fault, actor, put, great, perform, overal, stori, well, execut, movi, open, great, zinger, crazi, old, guy, forc, young, aborigin, girl, car, road, forc, endur, 40, minut, charact, develop, entir, new, group, charact, dont, know, 40, minut, turn, one, eventu, discov, girl, bodi, stori, progress, therebr, br, stori, pick, point, realli, goe, nowher, 2, hour, ask, point, see, charact, struggl, accus, racism, stupid, handl, discoveri, stori, ultim, unsatisfi, felt, unfinish, well, act, there, strong, enough, backbon, film, warrant, recommend]"
24925,"The first thing I thought when I saw this films was: It is not really a film, at least it is not what we imagine spontaneously when we hear the word ""film"". it is entirely symbolic, everything in it has a figurative meaning. So if you are not used to express thing in a symbolic way, you will find it strange, if you are not acquainted with philosophy, religion, spiritual life, you will think it's just a fairy-tale... and even a weird one, chaotic. For me ""The legend of Zu"" is perfectly transparent. And I do like it. It tells us in images the story about the fight between light and darkness, the fight that is as old as humanity, and every one who is in search of the sens in this life is confronted with it. The film is obviously made by Buddhists. I am not a Buddhist. My religion and the vision of the world and human is different. But as far as we are all humans and have the same human nature we necessarily have common experiences and can understand each other. It is a really beautiful film! And I which we had more films like this - films that have a meaning. There are too many empty stories which are good only to make time pass more quickly.","[first, thing, thought, saw, films, really, film, least, imagine, spontaneously, hear, word, film, entirely, symbolic, everything, figurative, meaning, used, express, thing, symbolic, way, find, strange, acquainted, philosophy, religion, spiritual, life, think, fairytale, even, weird, one, chaotic, legend, Zu, perfectly, transparent, like, tells, us, images, story, fight, light, darkness, fight, old, humanity, every, one, search, sens, life, confronted, film, obviously, made, Buddhists, Buddhist, religion, vision, world, human, different, far, humans, human, nature, necessarily, common, experiences, understand, really, b

Making Embeddings

When making the embeddings, we choose values of 4 for window and 1 for sg. 1 was chosen for sg to use SkipGram. The data has 40000 reviews, and even though this is sufficient to train a Word2Vec model, rare or domain-specific sentiment words may not be well captured by a CBOW. The window chosen was 4 because the sentiment is often localized in very few words.

In [26]:
reviews = list(X_train['Stem Words'])
model = Word2Vec(
    sentences=reviews,  # Tokenized data
    vector_size=100,             # Embedding dimensionality
    window=4,                    # Context window size
    min_count=2,                 # Minimum frequency for words to be included
    sg=1,                        # Use CBOW (set to 1 for Skip-Gram)
    workers=4,                   # Number of threads for faster training
    epochs=10                    # Number of passes over the data
)

# Save the model for future use
model.save("word2vec_cbow.model")

In [27]:
model.wv['film']

array([-0.0009694 ,  0.19847532, -0.08400735, -0.13118428,  0.40814745,
       -0.17674711,  0.01580076,  0.34279191, -0.2369714 , -0.24268194,
       -0.17975253, -0.53774995, -0.0188964 ,  0.32524678,  0.0413796 ,
       -0.057226  ,  0.13068002,  0.2682956 , -0.20714618, -0.24078478,
        0.15226355, -0.11574178,  0.16071515, -0.17458622,  0.03153089,
       -0.14883302, -0.33633927,  0.2563716 , -0.40459186,  0.08784578,
        0.31675297,  0.04309266, -0.13166523, -0.22508906, -0.12119685,
        0.46627486,  0.39056936, -0.0638577 ,  0.06832714,  0.02203305,
        0.29075542, -0.1774595 , -0.1122286 ,  0.01611961, -0.04348327,
        0.22308953, -0.01685513, -0.41038564,  0.0225801 , -0.09462918,
        0.18233944, -0.05162655, -0.02161762, -0.03642436, -0.09965254,
        0.15291524,  0.21992181, -0.2658345 , -0.18769139,  0.2091243 ,
       -0.01474115, -0.00814192,  0.15608463,  0.10830636, -0.23578224,
        0.21923047,  0.15001787,  0.18081728, -0.20508568,  0.20

In [28]:
model.wv.most_similar('film')

[('movi', 0.8962374329566956),
 ('shortfilm', 0.7833355665206909),
 ('telemovi', 0.760982871055603),
 ('filmit', 0.7597492933273315),
 ('towelhead', 0.7572776079177856),
 ('cassavettess', 0.7563760280609131),
 ('filmbr', 0.7530413269996643),
 ('generalis', 0.748876690864563),
 ('directtodvd', 0.7481429576873779),
 ('swordsandsorceri', 0.7469770908355713)]

'film' and 'movi' (movie) have similar vectors indicating this embedding seems to have worked

In [29]:
def transform_review_to_vector(review, model): #Create 1 vector of the text based on the Word2Vec vectors of the words in the sentence
    word_vectors = []
    for word in review:
        if word in model.wv:
            word_vectors.append(model.wv[word])
    if len(word_vectors) == 0:  # Handle case where no words are in vocabulary
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

## 4. Creating our FNN

Developping models: Embeddings, Features, Embeddings + Features. The architecture chosen starts at a high number of neurons, and then goes down gradually over the layers, so that it extracts more and more high level features. The data is balanced (same ratio for positives and negatives), so we use accuracy for model selection. The number of neurons for the model with only features was reduced (8 -> 4 -> 2)

In [30]:
X_train.shape

(32000, 7)

In [31]:
def prepare_data(choice, data, cols, model):
    if choice["Embedding"] == 1:
      data["Embedded Data"] = data["Stem Words"].apply(lambda x : transform_review_to_vector(x, model))
      df_embeddings = pd.DataFrame(data['Embedded Data'].tolist(), index=data.index)
    else:
      df_embeddings = pd.DataFrame()

    if choice["Features"] == 1:
      df_features = data[cols]
    else:
      df_features = pd.DataFrame()

    final_df = pd.concat([df_embeddings, df_features], axis=1)
    return final_df.to_numpy()
#Feedforward Neural Network (FNN)
choice_params = [{"Embedding" : 1, "Features" : 0}, {"Embedding" : 0, "Features" : 1}, {"Embedding" : 1, "Features" : 1}]
network_sizes = {(0, 1): (8, 4, 2)} #if we only use the features (and not the embedding), we have a smaller model
results = {}
for option in choice_params:
  X_train_opt = prepare_data(option, X_train, cols, model)
  X_val_opt = prepare_data(option, X_val, cols, model)
  X_test_opt = prepare_data(option, X_test, cols, model)


  # Input size depends on your feature representation (e.g., TF-IDF size or embedding size)
  input_dim = X_train_opt.shape[1]
  tuple_option = (option["Embedding"], option["Features"])
  if tuple_option in network_sizes.keys():
      # Define the FNN
    fnn = Sequential([
      Dense(network_sizes[tuple_option][0], activation='relu', input_shape=(input_dim,)),  # Input layer
      Dropout(0.5),  # Dropout for regularization
      Dense(network_sizes[tuple_option][1], activation='relu'),  # Hidden layer
      Dropout(0.4),
      Dense(network_sizes[tuple_option][2], activation = 'relu'),
      Dropout(0.3),
      Dense(1, activation='sigmoid')  # Output layer for binary classification
      ])
  else:
    fnn = Sequential([
      Dense(128, activation='relu', input_shape=(input_dim,)),  # Input layer
      Dropout(0.5),  # Dropout for regularization
      Dense(64, activation='relu'),  # Hidden layer
      Dropout(0.4),
      Dense(32, activation = 'relu'),
      Dropout(0.3),
      Dense(1, activation='sigmoid')  # Output layer for binary classification
      ])



      # Compile the model
  fnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
  history = fnn.fit(X_train_opt, y_train, validation_data=(X_val_opt, y_val), batch_size = 64, epochs=50, callbacks=[early_stopping])
  last_val_accuracy = history.history['val_accuracy'][-1]
  results[tuple_option] = last_val_accuracy

Epoch 1/50


C:\Users\kkasp\anaconda3\envs\Text_mining\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.6860 - loss: 0.5611 - val_accuracy: 0.8666 - val_loss: 0.3264
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8457 - loss: 0.3737 - val_accuracy: 0.8677 - val_loss: 0.3172
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.8534 - loss: 0.3549 - val_accuracy: 0.8730 - val_loss: 0.3065
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8562 - loss: 0.3483 - val_accuracy: 0.8687 - val_loss: 0.3097
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.8560 - loss: 0.3459 - val_accuracy: 0.8717 - val_loss: 0.3039
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.8575 - loss: 0.3409 - val_accuracy: 0.8749 - val_loss: 0.3036
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8604 - loss: 0.3397 - val_accuracy: 0.8733 - val_loss: 0.3167
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8612 - loss: 0.3347 - val_accuracy: 0.8752 - v

C:\Users\kkasp\anaconda3\envs\Text_mining\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5674 - loss: 0.6787 - val_accuracy: 0.7286 - val_loss: 0.6136
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6005 - loss: 0.6510 - val_accuracy: 0.7344 - val_loss: 0.5927
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6184 - loss: 0.6355 - val_accuracy: 0.7350 - val_loss: 0.5870
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6173 - loss: 0.6340 - val_accuracy: 0.7352 - val_loss: 0.5784
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6271 - loss: 0.6262 - val_accuracy: 0.7383 - val_loss: 0.5747
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6322 - loss: 0.6218 - val_accuracy: 0.7356 - val_loss: 0.5728
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6299 - loss: 0.6233 - val_accuracy: 0.7374 - val_loss: 0.5674
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6322 - loss: 0.6256 - val_accuracy: 0.7393 - val_

C:\Users\kkasp\anaconda3\envs\Text_mining\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7109 - loss: 0.5527 - val_accuracy: 0.8504 - val_loss: 0.3483
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8375 - loss: 0.3813 - val_accuracy: 0.8687 - val_loss: 0.3190
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8482 - loss: 0.3601 - val_accuracy: 0.8668 - val_loss: 0.3109
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8576 - loss: 0.3459 - val_accuracy: 0.8733 - val_loss: 0.3079
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8642 - loss: 0.3314 - val_accuracy: 0.8735 - val_loss: 0.3038
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8589 - loss: 0.3350 - val_accuracy: 0.8742 - val_loss: 0.3036
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8651 - loss: 0.3248 - val_accuracy: 0.8751 - val_loss: 0.3010
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8638 - loss: 0.3288 - val_accuracy: 0.

In [32]:
results

{(1, 0): 0.8733749985694885,
 (0, 1): 0.7394999861717224,
 (1, 1): 0.8766250014305115}

(1, 0): The model is trained using only embeddings and achieves an accuracy of approximately 87.34%.

(0, 1): The model uses only the predefined feature columns and achieves an accuracy of approximately 73.95%.

(1, 1): The model combines both embeddings and feature columns and achieves an accuracy of approximately 87.66%.

The difference between the model with only embeddings and the model with embeddings and feature columns is rather small. However, the latter model performs best.

## 5. Results

In [33]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [34]:
y_pred_probs = fnn.predict(X_test_opt)  # Predicted probabilities
y_pred = (y_pred_probs > 0.5).astype(int)  # Convert probabilities to binary labels (0 or 1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print evaluation metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Precision: 0.8743728677503512
Recall: 0.8646556856519151
F1 Score: 0.869487128317701


We achieve a precision of 87.44%, a recall score of 86.47% and an F1 score of 86.95%. These high scores indicate our model is quite strong in predicting the sentiment of reviews based on the review text.